# Generate custom template library and recordings with models from the Allen Institute of Brain Science

In this notebook, we show how to build a custom template library using cell models from the Allen Institute database.

In particuar, we downloaded 3 cell models:

- Cell ID [488695444](https://celltypes.brain-map.org/experiment/electrophysiology/488695444) (spiny - excitatory)
- Cell ID [488680211](https://celltypes.brain-map.org/experiment/electrophysiology/488680211) (spiny - excitatory)
- Cell ID [487667205](https://celltypes.brain-map.org/experiment/electrophysiology/487667205) (aspiny - inhibitory)

To get the models, select `Select neuronal model` -> `Biophysical - perisomatic` (or `Biophysical - all active`, if available), download the zip file, and unzip it in a folder with the same name as the cell (e.g. neuronal_model_491623973) in the `allen_models` folder in the working directory. Note that the actual model id might be different than the cell id! In this notebook we assume that the created folder has the model ID (same ID as the downloaded zip).

In [ ]:
import numpy as np
import MEArec as mr
import MEAutility as mu
from pathlib import Path
import neuron
import LFPy
import os
import json
import matplotlib.pylab as plt
from pprint import pprint
%matplotlib notebook

## Generating custom templates

We need to define a function to load the cell models in LFPy. The following function takes the cell folder as input, and it returns an `LFPy.Cell` object. 
In order to be used in MEArec to load custom models, the function needs to have the following arguments:

- cell_model_folder: path to cell model folder (str)
- dt: sampling period in s (float)
- start_T: start time of NEURON simulation in s (default 0)
- end_T: end time of NEURON simulation in s (default 1)

In [ ]:
# Function to load Allen cells in LFPy
def return_allen_cell(cell_model_folder, dt=2**-5, start_T=0, end_T=1):    
    cell_model_folder = Path(cell_model_folder)
    cwd = os.getcwd()
    os.chdir(cell_model_folder)
    
    # compile mechanisms
    mod_folder = "modfiles"
    os.chdir(mod_folder)
    os.system('nrnivmodl')
    os.chdir('..')
    neuron.load_mechanisms(mod_folder)
    params = json.load(open("fit_parameters.json", 'r'))

    celsius = params["conditions"][0]["celsius"]
    reversal_potentials = params["conditions"][0]["erev"]
    v_init = params["conditions"][0]["v_init"]
    active_mechs = params["genome"]
    neuron.h.celsius = celsius

    cell_parameters = {
        'morphology': 'reconstruction.swc',
        'v_init': v_init,  # initial membrane potential
        'passive': False,  # turn on NEURONs passive mechanism for all sections
        'nsegs_method': 'lambda_f',  # spatial discretization method
        'lambda_f': 200.,  # frequency where length constants are computed
        'dt': dt,  # simulation time step size
        'tstart': start_T,  # start time of simulation, recorders start at t=0
        'tstop': end_T,  # stop simulation at 100 ms.
    }

    cell = LFPy.Cell(**cell_parameters)

    for sec in neuron.h.allsec():
        sec.insert("pas")
        sectype = sec.name().split("[")[0]
        for sec_dict in active_mechs:
            if sec_dict["section"] == sectype:
                # print(sectype, sec_dict)
                if not sec_dict["mechanism"] == "":
                    sec.insert(sec_dict["mechanism"])
                exec ("sec.{} = {}".format(sec_dict["name"], sec_dict["value"]))

        for sec_dict in reversal_potentials:
            if sec_dict["section"] == sectype:
                # print(sectype, sec_dict)
                for key in sec_dict.keys():
                    if not key == "section":
                        exec ("sec.{} = {}".format(key, sec_dict[key]))
    
    os.chdir(cwd)

    return cell

Let's also define a convenient function to plot the projections of the loaded cell.

In [ ]:
def plot_cell_projections(cell):
    fig = plt.figure()
    ax_xy = fig.add_subplot(2,2,1)
    ax_xz = fig.add_subplot(2,2,2)    
    ax_yz = fig.add_subplot(2,2,3)    
    
    for i, (xs, xe, ys, ye, zs, ze) in enumerate(zip(cell.xstart, cell.xend, cell.ystart, cell.yend, cell.zstart, cell.zend)):
        if i in cell.get_idx('soma'):
            ax_xy.plot([xs, xe], [ys, ye], color='k', lw=5)
            ax_xz.plot([xs, xe], [zs, ze], color='k', lw=5)
            ax_yz.plot([ys, ye], [zs, ze], color='k', lw=5)
        else:
            ax_xy.plot([xs, xe], [ys, ye], color='k')
            ax_xz.plot([xs, xe], [zs, ze], color='k')
            ax_yz.plot([ys, ye], [zs, ze], color='k')
        
    ax_xy.axis('equal')
    ax_xz.axis('equal')
    ax_yz.axis('equal')
    ax_xy.set_xlabel('x')
    ax_xy.set_ylabel('y')
    ax_xz.set_xlabel('x')
    ax_xz.set_ylabel('z')
    ax_yz.set_xlabel('y')
    ax_yz.set_ylabel('z')

    return fig

First, let's test that the cell model is loaded and run properly by our newly defined function.

In [ ]:
cell_folder = 'allen_models/neuronal_model_488462965/'

In [ ]:
cell = return_allen_cell(cell_folder)

In [ ]:
fig = plot_cell_projections(cell)

Great! The cell is loaded properly. Let's now test that the simulation is actually working.
In order to simulate a few spikes that will be used to simulate extracellular action potentials, we can use the `run_cell_model` function. By default, this function runs models from the Blue Brain Project repository, but we can use the `custom_return_cell_function` argument to load and simulate and arbitrary cell model. This function simulates the cell and can return the `LFPy.Cell` object, the somatic membrane potential, and the transmembrane currents for all compartments (when `save` is set to `False`.

We first need to retrieve some parameters for template generation:

In [ ]:
template_params = mr.get_default_templates_params()
template_params['seed'] = 0

In [ ]:
pprint(template_params)

In [ ]:
cell, v, i = mr.run_cell_model(cell_folder, verbose=True, save=False, 
                               custom_return_cell_function=return_allen_cell, 
                               **template_params)

Let's now plot the the somatic membrane potential and transmembrane current (the soma is compartment 0) for each spike:

In [ ]:
fig = plt.figure()
ax_v = fig.add_subplot(1,2,1)
ax_i = fig.add_subplot(1,2,2)
_ = ax_v.plot(v.T)
_ = ax_i.plot(i[:, 0].T)

### Simulating extracellular action potentials (EAPs)

We have to define other parameters for the extracellular simulation. In this case, we will randomly rotate the cells in 3D and generate 10 templates (at random locations) for each cell model.
Note that the `physrot` rotation is only implemented for BBP models, as it reproduces a specific physiological rotation depending on the cell type.

In [ ]:
template_params['rot'] = '3drot'
template_params['n'] = 10

We can choose the probe that we want to use. In this case we'll use the default `Neuronexus-32` probe, with 32 electrodes.

In [ ]:
print(template_params['probe'])

In order to list available probes we can use either the `MEAutility` package or the `MEArec` command line interface:

In [ ]:
print(mu.return_mea_list())

In [ ]:
!mearec available-probes

The `simulate_templates_one_cell` simulates and returns EAPs, locations of the soma, and rotations applied to each cell before computing the EAP.

In [ ]:
eaps, locs, rots = mr.simulate_templates_one_cell(cell_folder, 
                                                  intra_save_folder='allen_sim', params=template_params,
                                                  verbose=True, custom_return_cell_function=return_allen_cell)

The `eaps` have a shape of (n_templates, n_electrodes, n_timepoints). The `locs` and `rots` have a shape of (n_templates, 3).

In [ ]:
print(eaps.shape)
print(locs.shape)
print(rots.shape)

### Generating EAPs for all cell models and assembling a template library

We can now loop through all available cell models and build a template library. In order to do that, we also have to provide the cell type, that we can access from the `json` file.

In [ ]:
cell_models = [p for p in Path('allen_models/').iterdir()]
print(cell_models)

As we now the cell type of the different cells, let's build a dictionary to easily retrieve and save the cell type. At this point, we can choose how we want to characterize excitatory and inhibitory calls. Note that this information will need to be passed to the recording generation phase. For cells from the Allen Institute database we can use "spiny" for exctitatory and "aspiny" for inhibitory:

In [ ]:
cell_types = {'488462965': 'spiny', '489932682': 'spiny', '489932435': 'aspiny'}

Let's initialize some variables that will contain our EAPs, locations, rotations, and cell_types:

In [ ]:
templates, template_locations, template_rotations, template_celltypes = [], [], [], []

for cell in cell_models:
    eaps, locs, rots = mr.simulate_templates_one_cell(cell_folder, intra_save_folder='allen_sim', 
                                                      params=template_params, verbose=True, 
                                                      custom_return_cell_function=return_allen_cell)
    # find cell type
    cell_type = None
    for k, v in cell_types.items():
        if k in str(cell):
            cell_type = v
            break
    print("Cell", cell, "is", cell_type)
    
    # if first cell, initialize the arrays
    if len(templates) == 0:
        templates = eaps
        template_locations = locs
        template_rotations = rots
        template_celltypes = np.array([cell_type]*len(eaps))
    else:
        templates = np.vstack((templates, eaps))
        template_locations = np.vstack((template_locations, locs))
        template_rotations = np.vstack((template_rotations, rots))
        template_celltypes = np.concatenate((template_celltypes, np.array([cell_type]*len(eaps))))
    

In [ ]:
print(templates.shape)
print(template_locations.shape)
print(template_rotations.shape)
print(template_celltypes.shape)

We can now build a `TemplateGenerator` object that can be stored as an `h5` file and used to simulate recordings. 

We first need to create two dictionaries, `temp_dict` and `info`, containing the templates and related information.

In [ ]:
temp_dict = {'templates': templates, 
             'locations': template_locations, 
             'rotations': template_rotations,
             'celltypes': template_celltypes}
info = {}
info['params'] = template_params
info['electrodes'] = mu.return_mea_info(template_params['probe'])

Then we can instantiate a `TemplateGenerator` object. We can alsu use the `MEArec` built-in plot functions to inspect the templates:

In [ ]:
tempgen = mr.TemplateGenerator(temp_dict=temp_dict, info=info)

In [ ]:
mr.plot_templates(tempgen)

Finally, we can save the template library so that we can easily use it to build recordings:

In [ ]:
mr.save_template_generator(tempgen=tempgen, filename='allen/templates_allen.h5')

## Generating recordings

We can now use the Allen template library to assemble recordings. In order to do so, we have to change the recording parameters `cell_types` to tell the simulator that "spiny" means excitatory and "aspiny" means inhibitory cells:

In [ ]:
rec_params = mr.get_default_recordings_params()
pprint(rec_params)

In [ ]:
rec_params['cell_types'] = {'excitatory': ['spiny'], 'inhibitory': ['aspiny']}

Let's now simulate a 30-s recording, with 5 excitatory cells and 2 inhibitory cells. Since we have only 30 templates in total (it's advised to simulate many more templates per cell to be able to generate multiple different recordings), we will reduce the minimum distance between cells to 5um and use a minimum amplitude of 30uV.

In [ ]:
rec_params['spiketrains']['duration'] = 30
rec_params['spiketrains']['n_exc'] = 5
rec_params['spiketrains']['n_inh'] = 2
rec_params['templates']['min_dist'] = 5
rec_params['templates']['min_amp'] = 30

In [ ]:
recgen = mr.gen_recordings(params=rec_params, templates='allen/templates_allen.h5', verbose=True)

Let's now plot the generated spike trains, the selected templates, and the recordings!

In [ ]:
ax_st = mr.plot_rasters(recgen.spiketrains)
ax_temp = mr.plot_templates(recgen)
ax_rec = mr.plot_recordings(recgen, start_time=0, end_time=5, overlay_templates=True, lw=0.5)

Finally, we can save the generated recordings in `h5` format:

In [ ]:
mr.save_recording_generator(recgen=recgen, filename='allen/recordings_allen.h5')